# Diseño de la entrada de datos

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

imagen_alto = 128
imagen_ancho = 128
canales_de_color = 3

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(imagen_alto, imagen_ancho, canales_de_color)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

input_shape = tf.keras.Input(shape=(imagen_alto, imagen_ancho, canales_de_color))

c:\Users\achav\Documents\Osteoartritis\venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Arquitectura de la red

In [2]:
# Definir la arquitectura de la CNN
model = models.Sequential([
    (input_shape),  # Capa de entrada (128x128x1)
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # Capa de salida (2 neuronas, una por cada clase)
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,267,908 (12.47 MB)

 Trainable params: 3,267,908 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

# Importación de datos

In [6]:
import os
import cv2
import numpy as np
import pathlib
import matplotlib.pyplot as plt

data_dir = pathlib.Path('assets')

# Lista para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Clases disponibles
clases = ['class_A', 'class_B', 'class_C',]

# Recorrer las carpetas de las clases
for plate in os.listdir(data_dir):
    for clase in clases:
        for file in os.listdir(data_dir / plate / 'resized' / clase):
            # Leer la imagen
            imagen = cv2.imread(str(data_dir / plate / 'resized' / clase / file), cv2.IMREAD_COLOR)
            
            # Normalizar la imagen
            imagen = imagen / 255.0
            # Agregar la imagen a la lista
            imagenes.append(imagen)
            # Agregar la etiqueta a la lista en el formato [1, 0] o [0, 1]
            etiquetas.append([0, 1, 0] if clase == 'class_B' else [1, 0, 0])

# Convertir las listas de imágenes y etiquetas a arrays numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'assets\\296-19_001\\resized\\class_D'

# Alistamiento de los datos

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y validación
imagenes_entrenamiento, imagenes_validacion, etiquetas_entrenamiento, etiquetas_validacion = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Convertir las listas de imágenes y etiquetas de entrenamiento a arrays numpy
imagenes_entrenamiento = np.array(imagenes_entrenamiento)
etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)

# Convertir las listas de imágenes y etiquetas de validación a arrays numpy
imagenes_validacion = np.array(imagenes_validacion)
etiquetas_validacion = np.array(etiquetas_validacion)

# Entrenar el modelo

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

num_epochs = 12
batch_size = 32

# Directorio donde se guardarán los registros para TensorBoard
# directorio_logs = "logs/"

# Callback de TensorBoard
# tensorboard_callback = TensorBoard(log_dir=directorio_logs, histogram_freq=1)

# Entrenar el modelo con el callback de TensorBoard
model.fit(
    imagenes_entrenamiento, etiquetas_entrenamiento,
    epochs=num_epochs,
    batch_size=batch_size,
    )


# Evaluar el modelo

In [ ]:
test_loss, test_acc = model.evaluate(imagenes_validacion, etiquetas_validacion)
print('Precisión en datos de prueba:', test_acc)

# Guardado del entrenamiento

In [ ]:
model.save_weights('pesos_modelo_cnn.weights.h5')
model.save(filepath='modelo_cnn.keras')

In [ ]:
# Make a sound when done

import winsound
frequency = 880  # Set Frequency To 2500 Hertz

duration = 700  # Set Duration To 1000 ms == 1 second

winsound.Beep(frequency, duration)


# Cargar el modelo y los pesos

In [ ]:
# loaded_model = models.load_model('modelo_cnn.keras')

# loaded_model.load_weights('pesos_modelo_cnn.weights.h5')

# Cargar una imagen de prueba

In [ ]:
import os

OPENSLIDE_PATH = r'C:\Users\achav\Documents\Osteoartritis\openslide-win64\bin'

if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
        print('Using OpenSlide from', openslide.__file__)
else:
    import openslide

plate_name = "Ms-13-22-1_001"
plate_path = plate_name + ".svs"
path = f"assets/{plate_path}"

# Check if file exists
if not os.path.isfile(path):
    print(f"File {path} not found")

slide = openslide.OpenSlide(path)

# Get a thumbnail of the whole slide
thumbnail = slide.get_thumbnail((slide.dimensions[0] // 64, slide.dimensions[1] // 64))

# Convert the thumbnail to a numpy array
thumbnail = np.array(thumbnail)

# Display the thumbnail
plt.imshow(thumbnail)

plate_name = "Ms-13-22-1_001"
plate_path = plate_name + ".svs"
path = f"assets/{plate_path}"

# Check if file exists
if not os.path.isfile(path):
    print(f"File {path} not found")

slide = openslide.OpenSlide(path)

# Get a thumbnail of the whole slide
thumbnail = slide.get_thumbnail((slide.dimensions[0] // 64, slide.dimensions[1] // 64))

# Convert the thumbnail to a numpy array
thumbnail = np.array(thumbnail)

# Display the thumbnail
plt.imshow(thumbnail)

In [ ]:
from collections import defaultdict

from time import sleep

class_colors = {0: (255, 0, 0), 1: (0, 255, 0), 2: (0, 0, 255)}
level = 0

# Get the dimensions of the slide
width, height = slide.level_dimensions[level]

print(slide.level_dimensions)
print(slide.level_downsamples)
print(width, height)

# Create a count dict to store the number of tiles for each class
count = defaultdict(int)

original = np.array(slide.read_region((0, 0), level, (width, height)).convert("RGB"))

# Convert the image to a numpy array
# results = np.zeros((width, height, 3), dtype=np.uint8)

# print(results.shape)
for i in range(0, 1023, 1024):
    print(f"i: {i} / {height}")
    for j in range(0, 1023, 1024):
        print(f"j: {j} / {width}")
        try:
            
            image = slide.read_region((i, j), level, (256, 256)).convert("RGB")
            # Show thumbnail
            image = image.resize((128, 128))  # Ajustar al tamaño esperado por el modelo
            image = np.array(image)
            # plt.imshow(image)

            # Asegurarse de que la imagen tiene la forma correcta (128, 128, 3)
            image = np.expand_dims(image, axis=0)  # Añadir una dimensión para el batch
            # Hacer predicción con el modelo
            prediction = model.predict(image, verbose=0)
            print(prediction)
            # Obtener la clase con la mayor probabilidad
            prediction = np.argmax(prediction)
            # print(f"Prediction: {prediction}")
            
            # Pintar la región de la imagen original con el color de la clase predicha
            # cv2.rectangle(original, (i, j), (i + 256, j + 256), class_colors[prediction], 100)
            # cv2.rectangle(results, (i, j), (i + 256, j + 256), class_colors[prediction], -1)
            
            # Ajustar el tamaño del bloque a 4x4 en la matriz de resultados
            count[prediction] += 1

        except Exception as e:
            winsound.Beep(880, 1000)
            print(e)


        # Compare original image with the predicted image
        #plt.figure(figsize=(10, 10))
        #plt.subplot(1, 2, 1)
        #thumbnail = cv2.resize(original, (width // 64, height// 64))
        #plt.imshow(thumbnail)
        #plt.title("Original Image")
        #plt.axis("off")

        #plt.subplot(1, 2, 2)

        ## Resize the results to the same size as the original image
        #resultados = cv2.resize(results, (thumbnail.shape[1], thumbnail.shape[0]))

        #plt.imshow(resultados)
        #plt.title("Predicted Image")
        #plt.axis("off")

        #plt.show()
        

# Reduce in a factor of 4
# cv2.imwrite("original.png", cv2.resize(thumbnail, (width, height), interpolation=cv2.INTER_NEAREST))
# cv2.imwrite("results.png", cv2.resize(results, (width, height), interpolation=cv2.INTER_NEAREST))

winsound.Beep(880, 1000)
print(count)

In [ ]:
# Compare original image with the predicted image
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(thumbnail)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)

# Resize the results to the same size as the original image
results = cv2.resize(results, (thumbnail.shape[1], thumbnail.shape[0]))

plt.imshow(results)
plt.title("Predicted Image")
plt.axis("off")

plt.show()